<a href="https://colab.research.google.com/github/rryl/amazon_vine_analysis/blob/main/challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-15 22:24:38--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.17MB/s    in 0.8s    

2022-08-15 22:24:39 (1.17 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

Load Amazon Data into Spark Dataframe

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22480053|R28HBXXO1UEVJT|0843952016|      34858117|          The Rising|           Books|          5|            0|          0|   N|                N|Great Twist on Zo...|I've known about ...| 2012-05-03|
|         US|   44244451| RZKRFS2UUMFFU|031088926X|     676347131|Sticky Faith Teen...|           Books|          5|    

Create Dataframes to match tables.

In [5]:
from pyspark.sql.functions import to_date

In [6]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("", "customer_count")

In [7]:
customers_df.show(5)

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   47098686|                 1|
|    9973109|                 1|
|   52477231|                24|
|   27568383|                 1|
|   15519001|                 2|
+-----------+------------------+
only showing top 5 rows



In [8]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates()

In [9]:
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|1419701630|Chuck Close: Face...|
|0451216954|Dark Lover (Black...|
|0552142395|My Feudal Lord: A...|
|1931514941|   Love Hina, Vol. 1|
|0615462219|The Great Pain De...|
+----------+--------------------+
only showing top 5 rows



In [10]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

In [11]:
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R28HBXXO1UEVJT|   22480053|0843952016|      34858117| 2012-05-03|
| RZKRFS2UUMFFU|   44244451|031088926X|     676347131| 2012-05-03|
|R2WAU9MD9K6JQA|   20357422|0615268102|     763837025| 2012-05-03|
|R36SCTKYTVPZPC|   13235208|1900869225|     785539232| 2012-05-03|
|R10BM6JUOJX27Q|   26301786|1565129938|      64646125| 2012-05-03|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [12]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])

In [13]:
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R28HBXXO1UEVJT|          5|            0|          0|   N|                N|
| RZKRFS2UUMFFU|          5|           15|         15|   N|                Y|
|R2WAU9MD9K6JQA|          3|            6|          8|   N|                N|
|R36SCTKYTVPZPC|          5|           10|         11|   N|                Y|
|R10BM6JUOJX27Q|          3|            0|          0|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



Connect to the AWS RDS Instance and write each Dataframe.

In [14]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cryhoxymeyzu.us-east-1.rds.amazonaws.com:5432/challenge"
config = {"user":"postgres", 
          "password": "atlantys3", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table="review_id_table2", mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products_table2', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers_table2', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table2', mode=mode, properties=config)